# A1111 Model Installer — SDXL (UI)

Installs **SDXL** assets for Automatic1111 using a simple UI:

- **Checkpoints (paste URLs)** → `/workspace/a1111/models/Stable-diffusion/`
- **VAE (optional)** → `/workspace/a1111/models/VAE/`
- **ControlNet (SDXL)** → `/workspace/a1111/models/ControlNet/` (paste URLs for .safetensors)
- **Upscalers (ESRGAN)** → `/workspace/a1111/models/ESRGAN/`

Re-running is safe: existing files are skipped.

> For gated Hugging Face files, set `HF_TOKEN` in your RunPod template.

In [ ]:
# First time? Uncomment, run once, then Kernel → Restart
# !pip install -q ipywidgets tqdm requests huggingface_hub
print("Downloader UI ready (install dependencies once if widgets are missing).")

In [ ]:
from pathlib import Path
import os

# A1111 data root (matches WEBUI_ROOT in your image)
DATA_ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))
CKPT_DIR = DATA_ROOT / "models/Stable-diffusion"
VAE_DIR  = DATA_ROOT / "models/VAE"
CN_DIR   = DATA_ROOT / "models/ControlNet"
ESRGAN_DIR = DATA_ROOT / "models/ESRGAN"

for d in (CKPT_DIR, VAE_DIR, CN_DIR, ESRGAN_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("Data root:", DATA_ROOT)
print("↳ Checkpoints:", CKPT_DIR)
print("↳ VAE:", VAE_DIR)
print("↳ ControlNet (SDXL):", CN_DIR)
print("↳ Upscalers (ESRGAN):", ESRGAN_DIR)

## Helpers (with progress bars)
Supports **Hugging Face** (repo + filename) and **direct URLs**.

In [ ]:
import os, shutil, requests, shlex, subprocess, time
from tqdm import tqdm
from huggingface_hub import hf_hub_download
from pathlib import Path

def human(n: int) -> str:
    for unit in ["B","KB","MB","GB"]:
        if n < 1024:
            return f"{n:.1f} {unit}"
        n /= 1024
    return f"{n:.1f} TB"

def download_direct(url: str, dest: Path):
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest)
        return dest
    r = requests.get(url, stream=True)
    r.raise_for_status()
    total = int(r.headers.get('Content-Length', 0))
    tmp = dest.with_suffix(dest.suffix + ".part")
    with open(tmp, 'wb') as f, tqdm(total=total, unit='B', unit_scale=True, desc=dest.name, ncols=80) as bar:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk)
                bar.update(len(chunk))
    tmp.replace(dest)
    print("✔ Saved:", dest, f"({human(dest.stat().st_size)})")
    return dest

def download_hf(repo_id: str, filename: str, dest_dir: Path, dest_name: str=None, token: str=None):
    dest = dest_dir / (dest_name or filename)
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest)
        return dest
    print(f"→ HF: {repo_id}/{filename}")
    path = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        token=token or os.environ.get("HF_TOKEN"),
        local_dir=dest_dir,
        local_dir_use_symlinks=False,
        resume_download=True,
    )
    if Path(path) != dest:
        shutil.copy2(path, dest)
    print("✔ HF downloaded:", dest, f"({human(dest.stat().st_size)})")
    return dest

print("Helpers loaded.")

## Pick what to install
- Paste **SDXL checkpoints** (e.g., JuggernautXL, RealVisXL, DreamShaperXL) — one per line.
- **VAE**: commonly used **sdxl-vae-fp16-fix** (optional).
- **ControlNet SDXL**: paste URLs for `.safetensors` (canny/lineart/softedge/depth/openpose, etc.).
- **Upscaler**: 4x-UltraSharp (optional; works for SDXL & SD1.5).

In [ ]:
import ipywidgets as W
from IPython.display import display, Markdown

# --- Checkpoints (user URLs) ---
ckpt_text = W.Textarea(
    value="",
    placeholder="Paste one or more SDXL .safetensors URLs here (one per line).",
    layout=W.Layout(width="100%", height="140px")
)

# --- Optional VAE (SDXL VAE Fix) ---
vae_cb = W.Checkbox(description="Install SDXL VAE Fix (madebyollin/sdxl-vae-fp16-fix)", value=False)
vae_url = W.Text(
    value="https://huggingface.co/madebyollin/sdxl-vae-fp16-fix/resolve/main/sdxl_vae.safetensors",
    layout=W.Layout(width="100%")
)

# --- ControlNet SDXL (paste URLs) ---
cn_label = W.HTML("<b>ControlNet SDXL (.safetensors URLs)</b>")
cn_canny_cb = W.Checkbox(description="Canny", value=False)
cn_canny_url = W.Text(value="", placeholder="https://…/controlnet-xl-canny.safetensors", layout=W.Layout(width="100%"))
cn_soft_cb = W.Checkbox(description="SoftEdge", value=False)
cn_soft_url = W.Text(value="", placeholder="https://…/controlnet-xl-softedge.safetensors", layout=W.Layout(width="100%"))
cn_line_cb = W.Checkbox(description="LineArt", value=False)
cn_line_url = W.Text(value="", placeholder="https://…/controlnet-xl-lineart.safetensors", layout=W.Layout(width="100%"))
cn_depth_cb = W.Checkbox(description="Depth", value=False)
cn_depth_url = W.Text(value="", placeholder="https://…/controlnet-xl-depth.safetensors", layout=W.Layout(width="100%"))
cn_pose_cb = W.Checkbox(description="OpenPose", value=False)
cn_pose_url = W.Text(value="", placeholder="https://…/controlnet-xl-openpose.safetensors", layout=W.Layout(width="100%"))

# --- Upscaler (UltraSharp) ---
up_cb = W.Checkbox(description="4x-UltraSharp (ESRGAN)", value=True)
up_url = W.Text(
    value="https://huggingface.co/Bing-su/4x-UltraSharp/resolve/main/4x-UltraSharp.pth",
    layout=W.Layout(width="100%")
)

btn = W.Button(description="Download Selected", button_style="success", icon="download")
out = W.Output()

def on_click(_):
    out.clear_output()
    with out:
        errors = []

        # 1) VAE (optional)
        try:
            if vae_cb.value and vae_url.value.strip():
                download_direct(vae_url.value.strip(), VAE_DIR / "sdxl_vae.safetensors")
        except Exception as e:
            errors.append(("VAE", str(e)))

        # 2) ControlNet SDXL (direct URLs user-provided)
        cn_items = [
            (cn_canny_cb, cn_canny_url, "controlnet-xl-canny.safetensors"),
            (cn_soft_cb,  cn_soft_url,  "controlnet-xl-softedge.safetensors"),
            (cn_line_cb,  cn_line_url,  "controlnet-xl-lineart.safetensors"),
            (cn_depth_cb, cn_depth_url, "controlnet-xl-depth.safetensors"),
            (cn_pose_cb,  cn_pose_url,  "controlnet-xl-openpose.safetensors")
        ]
        for cb, urlw, fname in cn_items:
            try:
                if cb.value and urlw.value.strip():
                    download_direct(urlw.value.strip(), CN_DIR / fname)
            except Exception as e:
                errors.append((f"ControlNet {cb.description}", str(e)))

        # 3) Upscaler (UltraSharp)
        try:
            if up_cb.value and up_url.value.strip():
                download_direct(up_url.value.strip(), ESRGAN_DIR / "4x-UltraSharp.pth")
        except Exception as e:
            errors.append(("4x-UltraSharp", str(e)))

        # 4) Checkpoints (direct URLs)
        urls = [u.strip() for u in ckpt_text.value.splitlines() if u.strip()]
        for url in urls:
            try:
                name = url.split("/")[-1].split("?")[0]
                if not (name.endswith(".safetensors") or name.endswith(".ckpt")):
                    name += ".safetensors"
                download_direct(url, CKPT_DIR / name)
            except Exception as e:
                errors.append((url, str(e)))

        # Summary
        print("\n=== Summary ===")
        any_files = False
        for p in sorted(list(CKPT_DIR.glob('*')) + list(VAE_DIR.glob('*')) + list(CN_DIR.glob('*')) + list(ESRGAN_DIR.glob('*'))):
            try:
                print(f"✔ {p.name:45s} {os.path.getsize(p):,} B"); any_files = True
            except Exception:
                pass
        if not any_files:
            print("(No new files)")
        if errors:
            print("\nErrors:")
            for name, msg in errors:
                print("✖", name, "→", msg)

btn.on_click(on_click)

display(Markdown("### SDXL Downloads"))
display(Markdown("**Checkpoints (paste URLs):**"))
display(ckpt_text)

display(Markdown("**VAE (optional):**"))
display(vae_cb, vae_url)

display(cn_label)
display(cn_canny_cb, cn_canny_url)
display(cn_soft_cb, cn_soft_url)
display(cn_line_cb, cn_line_url)
display(cn_depth_cb, cn_depth_url)
display(cn_pose_cb, cn_pose_url)

display(Markdown("**Upscaler (ESRGAN):**"))
display(up_cb, up_url)

display(btn, out)

## (Optional) Restart the WebUI
Use this after new models are added so A1111 refreshes file lists.

In [ ]:
import subprocess, shlex, time
try:
    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"
    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True, timeout=20)
    if out.returncode == 0:
        print("✔ Restarted via supervisord. Wait ~10–15s, then refresh the WebUI (7860).\n")
        print(out.stdout.strip())
    else:
        print("Supervisorctl failed; trying process restart…")
        subprocess.run(["pkill", "-f", "launch.py"], check=False)
        time.sleep(5)
        print("✔ WebUI killed; supervisord will auto-restart. Refresh in ~10–15s.")
except Exception as e:
    print("Restart attempt hit an error:", e)
    print("Manual command:")
    print("supervisorctl -c /etc/supervisord.conf restart a1111")